In [1]:
#-- The current version is V1.1
#-- This is updated from v1.0 to V1.1
#-- The updated version is a refactored code of the previous version, and including the biographic data of the participants.
#-- Updated by Yusupha Ceesay on 29/07/2024
#-- Implemented on 01/08/2024
#-------------------------------------------------------------------------#

In [2]:
from redcap import Project
import pandas as pd
import datetime
today=pd.Timestamp.today()

#REDCap API configuration
##project configuration function
def project_configuration(api_url,api_key):
    return Project(api_url,api_key)

###indigo project configuration
project = project_configuration('https://ruff.mrc.gm:8443/redcap/api/', '63A451E2DA73C4D6A4C64A17B36B9565')

###Sensitization project configuration
sen_project = project_configuration('https://redcap.mrc.gm:8443/redcap/api/','008DBC63CA736CB68A3949DEFE43CC5D')


In [3]:
##Village mapping
village_mapping = {
'001':'Dumbuto',
'002':'Sankandi',
'003':'Nioro Jattaba',
'004':'Jattaba',
'005':'Jiffarong',
'006':'Bajana',
'007':'Kuli Kunda',
'008':'Jamaru',
'009':'Brikamanding',
'010':'Kantong Kunda',
'011':'Jali',
'013':'Manduar',
'014':'Bang Kuling',
'015':'Gissay',
'016':'Tankular',
'017':'Joli',
'018':'Kuyang',
'019':'Bantasu',
'020':'Santamba',
'021':'Missira',
'022':'Taborangkoto',
'023':'Burong',
'024':'Jula Kunda',
'025':'Karantaba',
'026':'Mandina',
'027':'Janneh Kunda',
'028':'Kemoto',
'029':'Keneba',
'030':'Batelling',
'031':'Sandeng',
'032':'Wudeba',
'034':'Kenokoto',
'035':'Manari',
'036':'Nineteen',
'040':'WUROKANG',
'041':'KWINELLA SANSANKONO',
'042':'KWINELLA NIA KUNDA',
'043':'TENDABA',
'044':'BUMARR',
'045':'BAMBAKO',
'046':'KUNDONG MARIAYA',
'047':'NEMA',
'048':'KUNDANG NUMU KUNDA',
'049':'KUNDANG FULA KUNDA',
'050':'NEMA KUTA',
'051':'JIRROFF',
'052':'MADINA ANGALLEH',
'053':'JATTA KUNDA',
'054':'MANDINA CENTRAL',
'055':'SARE SARJO',
'056':'SIBETO',
'057':'SARE NDALLA',
'058':'TABANANI',
'060':'WILLINGARA',
'061':'SARE MAMUDU'
}


In [4]:
# #Function to process sensitization data
# def process_sensitization_data(project,village_mapping):
#     fields = ['in_kiang_number', 'in_name', 'in_age', 'in_village', 'in_compound', 
#         'in_eden_number', 'in_contact1', 'in_contact2', 'in_contact3', 'in_contact4', 
#         'in_outcome', 'in_study_number'
#         ]
#     #sen_log_data= export_records_to_df(project,['indigo_sensitisation'],fields)
#     sen_log_data=project.export_records(forms=['indigo_sensitisation'])
#     sen_log_data = sen_log_data[(sen_log_data['in_outcome']=='1') &
#                                 (sen_log_data['in_kiang_number'].notnull()) &
#                                 (sen_log_data['in_study_number'].notnull())
#                                 ]
#     sen_log_data=sen_log_data.rename(columns={'in_study_number': 'con_participantid_q1'})
#     sen_log_data['in_village'] = sen_log_data['in_village'].map(village_mapping)

#     return sen_log_data

In [5]:
#Function to process sensitization data

def process_sensitization_data(project, village_mapping):
    fields = ['in_kiang_number', 'in_name', 'in_age', 'in_village', 'in_compound', 
              'in_eden_number', 'in_contact1', 'in_contact2', 'in_contact3', 'in_contact4', 
              'in_outcome', 'in_study_number']
    
    # Export records from the project
    sen_log_data = project.export_records(forms=['indigo_sensitisation'])
    
    # Convert to DataFrame if necessary
    if isinstance(sen_log_data, list):
        sen_log_data = pd.DataFrame(sen_log_data)

    # Filter the data
    sen_log_data = sen_log_data[(sen_log_data['in_outcome'] == '1') &
                                (sen_log_data['in_kiang_number'].notnull()) &
                                (sen_log_data['in_study_number'].notnull())]
    
    # Rename columns
    sen_log_data = sen_log_data.rename(columns={'in_study_number': 'con_participantid_q1'})
    
    # Map village names
    sen_log_data['in_village'] = sen_log_data['in_village'].map(village_mapping)

    return sen_log_data


In [6]:
#SENSITIZATION DATA
sen_log_data=process_sensitization_data(sen_project,village_mapping)

In [7]:

# Extract and convert data from Maternal Supplementation
MS_data = project.export_records(forms=['maternal_supplementation'])
MatSup_data = pd.DataFrame(MS_data)

# Extract and convert data from Randomization
Rand_data = project.export_records(forms=['randomization'])
Rand_data = pd.DataFrame(Rand_data)

# Select relevant columns and convert 'ran_date' to datetime
Rand_data = Rand_data[['con_participantid_q1', 'redcap_event_name', 'ran_date', 'rnd_arm']]
Rand_data['ran_date'] = pd.to_datetime(Rand_data['ran_date'], errors='coerce')

# Filter and map randomization data
rnd_group = {
    '1': 'B', '2': 'N', '3': 'P', '4': 'A', '5': 'X', '6': 'Y',
    '7': 'C', '8': 'U', '9': 'E', '10': 'K', '11': 'J', '12': 'H',
    '13': 'W', '14': 'T', '15': 'L'
}
rnd_data = Rand_data[(Rand_data['redcap_event_name'] == 'week_20_arm_1') & (Rand_data['rnd_arm'] != "")]
rnd_data['rnd_arm'] = rnd_data['rnd_arm'].map(rnd_group)

# Export infant ID and delivery data
#inf_dob = project.export_records(forms=['infantid', 'delivery'])
#inf_del = pd.DataFrame(inf_dob)

##Delivery Data
inf_dob = project.export_records(forms=['delivery'])
inf_del = pd.DataFrame(inf_dob)
inf_del['del_ddate_q7']=pd.to_datetime(inf_del['del_ddate_q7'], errors='coerce')

inf_del=inf_del[['con_participantid_q1','del_ddate_q7','redcap_event_name']]
inf_del=inf_del[(inf_del['redcap_event_name']=='delivery__postpart_arm_1') &
                        (inf_del['del_ddate_q7'].notnull())]

##Infant ID data
inf_dt= project.export_records(forms=['infantid'])
inf_dt = pd.DataFrame(inf_dt)
inf_dt=inf_dt[['con_participantid_q1', 'infantid','redcap_event_name']]
inf_dt=inf_dt[(inf_dt['redcap_event_name']=='baby_check_arm_2')]
##merge infant mother
inf_momther_dob = pd.merge(inf_dt, inf_del, on='con_participantid_q1', how='inner')
# Select and convert relevant columns
inf_data = inf_momther_dob[['con_participantid_q1', 'infantid', 'del_ddate_q7']]
df_infant = pd.DataFrame(inf_data)
df_infant['del_ddate_q7'] = pd.to_datetime(df_infant['del_ddate_q7'], errors='coerce')
df_infant['end_date'] = df_infant['del_ddate_q7'] + pd.DateOffset(months=6)

# Merge and select relevant columns for randomization data
Rand1 = rnd_data[['con_participantid_q1', 'ran_date', 'rnd_arm', 'redcap_event_name']]
Randomized_part = pd.merge(df_infant, Rand1, on='con_participantid_q1', how='right')
randomised = Randomized_part[['con_participantid_q1', 'infantid', 'ran_date', 'rnd_arm', 'del_ddate_q7', 'end_date']]

##WITHDRAWALS
#extract study termination data
termination_data = project.export_records(forms=['study_termination'])
term_data=pd.DataFrame(termination_data)
df_withdrw=term_data[['con_participantid_q1','st_date','st_reason','redcap_event_name']]
df_withdrw=pd.DataFrame(df_withdrw)

df_withdrw['st_date'] = pd.to_datetime(df_withdrw['st_date'], errors='coerce')

df_withdrawals = df_withdrw[
    (df_withdrw['redcap_event_name'] == 'end_of_study_arm_1') &
    (df_withdrw['st_date'].notnull())
]


## EXCLUDE ALL WITHDRAWALS FROM THE DATA

# Merge the dataframes on 'participant_id'
ran_con_merged_df = pd.merge(randomised, df_withdrawals, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
randomised= randomised[~randomised['con_participantid_q1'].isin(ran_con_merged_df['con_participantid_q1'])]




# Create DataFrame for randomised data and save to CSV
# randomised_df = pd.DataFrame({
#     'Participant ID': randomised['con_participantid_q1'],
#     'Infant ID': randomised['infantid'],
#     'Randomization date': randomised['ran_date'],
#     'Randomization Arm': randomised['rnd_arm'],
#     'Infant DoB': randomised['del_ddate_q7'],
#     'Supplementation end date': randomised['end_date'],
# })
# randomised_df.to_csv('Supplementation_reportT.csv', index=False)

# # Extract and convert data from Maternal Supplementation
# MS_1_dataT = MatSup_data[['con_participantid_q1', 'redcap_event_name', 'redcap_repeat_instrument', 'redcap_repeat_instance', 'ms_dvisit_q1', 'ms_datetaken', 'ms_sup_administered_q2']]
# MS_1_dataT['ms_datetaken'] = pd.to_datetime(MS_1_dataT['ms_datetaken'], errors='coerce')
# MS_1_dataT['ms_dvisit_q1'] = pd.to_datetime(MS_1_dataT['ms_dvisit_q1'], errors='coerce')
# MS_1_dataT = MS_1_dataT[MS_1_dataT['redcap_repeat_instance'] != '']
# MS_1_dataT['redcap_repeat_instance'] = MS_1_dataT['redcap_repeat_instance'].astype(int)

# # Find the minimum instance number for each ID and merge with the data
# min_instancesT = MS_1_dataT.groupby('con_participantid_q1')['redcap_repeat_instance'].min().reset_index()
# MS_first_instanceT = pd.merge(MS_1_dataT, min_instancesT, on=['con_participantid_q1', 'redcap_repeat_instance'], how='inner')

# # Select and convert relevant columns for Maternal Supplementation data
# MS_data = MatSup_data[['con_participantid_q1', 'redcap_event_name', 'redcap_repeat_instrument', 'ms_sup_administered_q2', 'ms_reason_not_admin_q2a', 'ms_takenafterfast', 'ms_datetaken', 'ms_completed_by']]
# MS_data['ms_datetaken'] = pd.to_datetime(MS_data['ms_datetaken'], errors='coerce')

# # Filter data based on supplementation taken
# MS_taken = MS_data[(MS_data['ms_sup_administered_q2'] == "1") | (MS_data['ms_takenafterfast'] == "1")]

# # Calculate the sum of all IP consumed by each participant
# sum_ms_taken = MS_taken.groupby('con_participantid_q1').size().reset_index(name='total_ms_taken')

# # Filter data based on supplementation not taken
# MS_not_taken = MS_data[((MS_data['ms_sup_administered_q2'] == "0") & (MS_data['ms_reason_not_admin_q2a'] != "6")) | ((MS_data['ms_reason_not_admin_q2a'] == "6") & (MS_data['ms_takenafterfast'] == "0"))]

# # Calculate the sum of all missed IPs for each participant
# sum_ms_not_taken = MS_not_taken.groupby('con_participantid_q1').size().reset_index(name='total_ms_not_taken')

# # Merge necessary dataframes for the final report
# rand_supp = rnd_data.merge(sum_ms_taken, on='con_participantid_q1', how='left')
# ms_rnd_merged = rand_supp.merge(sum_ms_not_taken, on='con_participantid_q1', how='left')
# ms_rnd_merged = ms_rnd_merged.merge(MS_first_instanceT, on='con_participantid_q1', how='left')

# # Select and create DataFrame for the IP log and save to CSV
# ms_rnd_log = ms_rnd_merged[['con_participantid_q1', 'rnd_arm', 'ms_dvisit_q1', 'total_ms_taken', 'total_ms_not_taken']]
# IP_log = pd.DataFrame({
#     'Participant ID': ms_rnd_log['con_participantid_q1'],
#     'Randomization Arm': ms_rnd_log['rnd_arm'],
#     'First visit date': ms_rnd_log['ms_dvisit_q1'],
#     'IP Consumed': ms_rnd_log['total_ms_taken'],
#     'IP Not Consumed': ms_rnd_log['total_ms_not_taken'],
# })
# IP_log.to_csv('IP_LogT.csv', index=False)


c:\Users\yuceesay\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
# Function to merge with sensitization data
def merge_with_sensitization(sup_df, sensitization_df):
    return pd.merge(sup_df, sensitization_df, on='con_participantid_q1', how='left')


In [9]:
# Merge with sensitization data
df_randomised = merge_with_sensitization(randomised, sen_log_data)

In [10]:

randomised_part= pd.DataFrame({
        'PARTICIPANT ID': df_randomised['con_participantid_q1'],
        'INFANT ID': df_randomised['infantid'],
        'KIANG NUMBER': df_randomised['in_kiang_number'],
        'NAME': df_randomised['in_name'],
        'VILLAGE': df_randomised['in_village'],
        'COMPOUND': df_randomised['in_compound'],
        'CONTACT1': df_randomised['in_contact1'],
        'CONTACT2': df_randomised['in_contact2'],
        'CONTACT3': df_randomised['in_contact3'],
        'CONTACT4': df_randomised['in_contact4'],
        'Randomization Date': df_randomised['ran_date'],
        'Randomization Arm': df_randomised['rnd_arm'],
       'Infant DoB': df_randomised['del_ddate_q7'],
    'Supplementation end date': df_randomised['end_date'],
})



In [11]:
# Function to save DataFrame to CSV with a timestamp
def save_to_csv(df, base_filename):
    now = datetime.datetime.now()
    timestamp = now.strftime('%Y%m%d_%H%M%S')
    filename = f'{base_filename}_{timestamp}.csv'
    df.to_csv(filename, index=False)


In [12]:
# Save the to a CSV file
save_to_csv(randomised_part, 'SupplementatonReport_v1.1')